In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
from transformers import DataCollatorForLanguageModeling
from datasets import load_dataset
from peft import LoraConfig, get_peft_model, TaskType
from trl import SFTTrainer
import wandb
import os

/home/miguel/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
EPOCHS = 20

In [3]:
model_name = "EleutherAI/pythia-70m"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained(model_name, device_map='auto') # load_in_4bit=True
print(model)

# Configuración de LoRA
lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM, 
    r=8,  # rango de LoRA
    lora_alpha=32,  # hiperparámetro de LoRA
    lora_dropout=0.1,  # dropout de LoRA
    #target_modules=[
    #    "q_proj",
    #    "k_proj",
    #    "v_proj",
    #    "o_proj",
    #    "gate_proj",
    #    "up_proj",
    #    "down_proj",
    #],
    target_modules=["query_key_value","dense","dense_h_to_4h","dense_4h_to_h"],
    #target_modules=['q_proj', 'v_proj', 'k_proj', 'o_proj', 'gate_proj', 'up_proj', 'down_proj']  # módulos objetivo para aplicar LoRA
)

model = get_peft_model(model, lora_config)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


GPTNeoXForCausalLM(
  (gpt_neox): GPTNeoXModel(
    (embed_in): Embedding(50304, 512)
    (emb_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-5): 6 x GPTNeoXLayer(
        (input_layernorm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (post_attention_dropout): Dropout(p=0.0, inplace=False)
        (post_mlp_dropout): Dropout(p=0.0, inplace=False)
        (attention): GPTNeoXSdpaAttention(
          (rotary_emb): GPTNeoXRotaryEmbedding()
          (query_key_value): Linear(in_features=512, out_features=1536, bias=True)
          (dense): Linear(in_features=512, out_features=512, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (mlp): GPTNeoXMLP(
          (dense_h_to_4h): Linear(in_features=512, out_features=2048, bias=True)
          (dense_4h_to_h): Linear(in_features=2048, out_features=512, bias=True)
        

In [4]:
# Preparación del dataset tipo 'instruct'
train_dataset_instruct = load_dataset('json', data_files='./data/arithmatic_expressions.json', split="train")
def tokenize_function_instruct(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True, max_length=64)

tokenized_train_dataset = train_dataset_instruct.map(tokenize_function_instruct, batched=True, remove_columns=['text'])

In [5]:
training_args = TrainingArguments(
    output_dir="./results",
    overwrite_output_dir=True,
    num_train_epochs=5,
    per_device_train_batch_size=2,
    save_steps=10_000,
    save_total_limit=2,
    logging_steps=100,
    learning_rate=1e-4,
    fp16=True,
    evaluation_strategy="steps",
    eval_steps=10_000,
    report_to="wandb"
)
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

trainer_instruct = SFTTrainer(
    model=model,
    train_dataset=tokenized_train_dataset,
    # eval_dataset=valid_dataset_mapped,   
    # dataset_text_field="text",
    max_seq_length=64,
    tokenizer=tokenizer,
    args=training_args,
    # compute_metrics=compute_metrics_fn,
    packing=True,
    data_collator=data_collator,
)

/home/miguel/.local/lib/python3.10/site-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/home/miguel/.local/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': max_seq_length, packing. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/home/miguel/.local/lib/python3.10/site-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/home/miguel/.local/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:192: UserWarning: You passed a `packing` argument to the SFTTrainer, the value yo

In [6]:
trainer_instruct.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: miguel_kjh. Use `wandb login --relogin` to force relogin


Step,Training Loss,Validation Loss


TrainOutput(global_step=5000, training_loss=3.359370947265625, metrics={'train_runtime': 99.8497, 'train_samples_per_second': 100.151, 'train_steps_per_second': 50.075, 'total_flos': 173046497280000.0, 'train_loss': 3.359370947265625, 'epoch': 5.0})

In [7]:
# Guardar el modelo fine-tuneado
trainer_instruct.save_model("./fine_tuned_model_math_pythia")
tokenizer.save_pretrained("./fine_tuned_model_math_pythia")

('./fine_tuned_model_math_pythia/tokenizer_config.json',
 './fine_tuned_model_math_pythia/special_tokens_map.json',
 './fine_tuned_model_math_pythia/tokenizer.json')